# Elo Score Calculation

In [1]:
import re
import os
import sys
from collections import defaultdict

In [2]:
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import xlrd
from labellines import labelLine, labelLines

In [4]:
# setting path
sys.path.append('../../src')

In [5]:
from eloscore import eloscore

In [6]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

## Summary

- Notebook that calcuates the Elo Score of mice competing over access to a tone associated reward port
- The data is a spreadsheet of combination of subjects (as rows) against dates (as columns)

## Importing Data for One Sheet

- The original data has been trimmed so that only the cells with dates, winner ID's, and relevant metadata were kept

In [7]:
xls = pd.ExcelFile("./data/Reward_Competition_Scoring_MasterList.xlsx")
sheets = xls.sheet_names

In [8]:
sheets

['Master Assignment List',
 'Ezra',
 'Albert',
 'Isabel',
 'Catalina',
 'Jocelyn',
 'Meghan',
 'Erika',
 'Naeliz']

In [9]:
reward_competition_df = pd.read_excel("./data/Reward_Competition_Scoring_MasterList.xlsx", sheet_name=sheets[0], header=0)

In [55]:
copy_reward_competition_df = reward_competition_df.copy()

In [56]:
copy_reward_competition_df.head()

,Date,Cage,Box,Match,Scorer,Trial 1 Time,Trial 1 Winner,Trial 2 Time,Trial 2 Winner,Trial 3 Time,...,Trial 17 Winner,Trial 18 Time,Trial 18 Winner,Trial 19 Time,Trial 19 Winner,Trial 20 Time,Trial 20 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties
0,2022-07-28,1,1,1v2,Naeliz,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-28,1,2,3v4,Erika,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-28,1,3,2v3,Ezra,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-28,1,4,1v4,Isabel,NaN,0:37-0:44,1.0,1:31-1:38,1.0,...,16:43-16:49,4.0,18:02-18:09,1.0,20:36-20:43,1.0,18.0,1.0,NaN,NaN
4,2022-07-28,1,1,2v4,Catalina,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
copy_reward_competition_df.columns

Index(['Date', 'Cage', 'Box', 'Match', 'Scorer', 'Trial 1 Time',
       'Trial 1 Winner', 'Trial 2 Time ', 'Trial 2 Winner', 'Trial 3 Time',
       'Trial 3 Winner', 'Trial 4 Time', 'Trial 4 Winner', 'Trial 5 Time',
       'Trial 5 Winner', 'Trial 6 Time', 'Trial 6 Winner', 'Trial 7 Time',
       'Trial 7 Winner', 'Trial 8 Time', 'Trial 8 Winner', 'Trial 9 Time',
       'Trial 9 Winner', 'Trial 10 Time ', 'Trial 10 Winner', 'Trial 11 Time',
       'Trial 11 Winner', 'Trial 12 Time ', 'Trial 12 Winner', 'Trial 13 Time',
       'Trial 13 Winner', 'Trial 14 Time', 'Trial 14 Winner', 'Trial 15 Time',
       'Trial 15 Winner', 'Trial 16 Time ', 'Trial 16 Winner', 'Trial 17 Time',
       'Trial 17 Winner', 'Trial 18 Time', 'Trial 18 Winner', 'Trial 19 Time',
       'Trial 19 Winner', 'Trial 20 Time', 'Trial 20 Winner', 'Mouse 1 Wins',
       'Mouse 2 Wins', 'Ties '],
      dtype='object')

In [58]:
copy_reward_competition_df = copy_reward_competition_df.dropna(subset=["Trial 1 Winner"])

## Removing Unnecessary Columns

In [59]:
copy_reward_competition_df.columns

Index(['Date', 'Cage', 'Box', 'Match', 'Scorer', 'Trial 1 Time',
       'Trial 1 Winner', 'Trial 2 Time ', 'Trial 2 Winner', 'Trial 3 Time',
       'Trial 3 Winner', 'Trial 4 Time', 'Trial 4 Winner', 'Trial 5 Time',
       'Trial 5 Winner', 'Trial 6 Time', 'Trial 6 Winner', 'Trial 7 Time',
       'Trial 7 Winner', 'Trial 8 Time', 'Trial 8 Winner', 'Trial 9 Time',
       'Trial 9 Winner', 'Trial 10 Time ', 'Trial 10 Winner', 'Trial 11 Time',
       'Trial 11 Winner', 'Trial 12 Time ', 'Trial 12 Winner', 'Trial 13 Time',
       'Trial 13 Winner', 'Trial 14 Time', 'Trial 14 Winner', 'Trial 15 Time',
       'Trial 15 Winner', 'Trial 16 Time ', 'Trial 16 Winner', 'Trial 17 Time',
       'Trial 17 Winner', 'Trial 18 Time', 'Trial 18 Winner', 'Trial 19 Time',
       'Trial 19 Winner', 'Trial 20 Time', 'Trial 20 Winner', 'Mouse 1 Wins',
       'Mouse 2 Wins', 'Ties '],
      dtype='object')

In [60]:
copy_reward_competition_df = copy_reward_competition_df.drop(["Mouse 1 Wins", "Mouse 2 Wins", "Ties "], axis=1)

In [61]:
columns_to_keep = [col for col in copy_reward_competition_df.columns if "winner" not in col.lower().strip()]

In [62]:
columns_to_keep

['Date',
 'Cage',
 'Box',
 'Match',
 'Scorer',
 'Trial 1 Time',
 'Trial 2 Time ',
 'Trial 3 Time',
 'Trial 4 Time',
 'Trial 5 Time',
 'Trial 6 Time',
 'Trial 7 Time',
 'Trial 8 Time',
 'Trial 9 Time',
 'Trial 10 Time ',
 'Trial 11 Time',
 'Trial 12 Time ',
 'Trial 13 Time',
 'Trial 14 Time',
 'Trial 15 Time',
 'Trial 16 Time ',
 'Trial 17 Time',
 'Trial 18 Time',
 'Trial 19 Time',
 'Trial 20 Time']

In [68]:
copy_reward_competition_df = copy_reward_competition_df[columns_to_keep]

## Add both IDs as a column

In [69]:
copy_reward_competition_df["animal_ids"] = copy_reward_competition_df["Match"].apply(lambda x: set(x.split("v")))

/tmp/ipykernel_1409/1948837574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_reward_competition_df["animal_ids"] = copy_reward_competition_df["Match"].apply(lambda x: set(x.split("v")))


In [70]:
copy_reward_competition_df

,Date,Cage,Box,Match,Scorer,Trial 1 Time,Trial 2 Time,Trial 3 Time,Trial 4 Time,Trial 5 Time,...,Trial 12 Time,Trial 13 Time,Trial 14 Time,Trial 15 Time,Trial 16 Time,Trial 17 Time,Trial 18 Time,Trial 19 Time,Trial 20 Time,animal_ids
0,2022-07-28,1,4,1v4,Isabel,NaN,1.0,1.0,1.0,1,...,1,1.0,1.0,1,1.0,1,4.0,1.0,1.0,"{4, 1}"
1,2022-07-28,2,4,3v4,Naeliz,NaN,3.0,4.0,4.0,3,...,3,4.0,4.0,3,3.0,3,3.0,3.0,4.0,"{3, 4}"
2,2022-07-28,2,3,2v4,Isabel,NaN,2.0,2.0,2.0,2,...,4,2.0,4.0,4,2.0,4,4.0,2.0,2.0,"{2, 4}"
3,2022-07-28,3,2,1v3,Isabel,NaN,1.0,1.0,1.0,3,...,1,3.0,1.0,1,1.0,1,1.0,1.0,3.0,"{3, 1}"
4,2022-07-28,4,3,1v2,Meghan,NaN,1.0,2.0,2.0,1,...,2,1.0,2.0,2,2.0,2,1.0,2.0,1.0,"{2, 1}"
5,2022-07-28,4,1,2v3,Meghan,NaN,3.0,3.0,3.0,tie,...,tie,2.0,2.0,2,3.0,tie,2.0,2.0,2.0,"{3, 2}"
6,2022-07-28,4,2,1v4,Meghan,NaN,4.0,1.0,4.0,4,...,1,4.0,4.0,4,1.0,1,4.0,1.0,1.0,"{4, 1}"
7,2022-07-28,5,3,2v3,Isabel,NaN,3.0,3.0,3.0,3,...,2,2.0,2.0,tie*,2.0,2,2.0,2.0,2.0,"{3, 2}"
8,2022-07-28,6,2,1v4,Isabel,NaN,1.0,1.0,1.0,1,...,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,"{4, 1}"
9,2022-07-29,1,3,1v3,Naeliz,NaN,1.0,3.0,1.0,1,...,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,"{3, 1}"


## Melt 

In [71]:
copy_reward_competition_df = copy_reward_competition_df.reset_index()

In [72]:
for index, row in copy_reward_competition_df.iterrows():
    print(row)
    break

index                               0
Date              2022-07-28 00:00:00
Cage                                1
Box                                 4
Match                             1v4
Scorer                         Isabel
Trial 1 Time                      NaN
Trial 2 Time                      1.0
Trial 3 Time                      1.0
Trial 4 Time                      1.0
Trial 5 Time                        1
Trial 6 Time                        1
Trial 7 Time                      1.0
Trial 8 Time                      1.0
Trial 9 Time                        1
Trial 10 Time                       1
Trial 11 Time                     1.0
Trial 12 Time                       1
Trial 13 Time                     1.0
Trial 14 Time                     1.0
Trial 15 Time                       1
Trial 16 Time                     1.0
Trial 17 Time                       1
Trial 18 Time                     4.0
Trial 19 Time                     1.0
Trial 20 Time                     1.0
animal_ids  

In [73]:
melted_reward_competition_df = copy_reward_competition_df.melt(id_vars=["index", "Date", "Cage", "Box", "Match", "Scorer", "animal_ids"], 
        var_name="Trial", 
        value_name="Winner")

In [67]:
melted_reward_competition_df

,index,Date,Cage,Box,Match,Scorer,animal_ids,Trial,Winner
0,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 1 Time,NaN
1,7,2022-07-28,2,4,3v4,Naeliz,"{3, 4}",Trial 1 Time,NaN
2,10,2022-07-28,2,3,2v4,Isabel,"{2, 4}",Trial 1 Time,NaN
3,17,2022-07-28,3,2,1v3,Isabel,"{3, 1}",Trial 1 Time,NaN
4,18,2022-07-28,4,3,1v2,Meghan,"{2, 1}",Trial 1 Time,NaN
...,...,...,...,...,...,...,...,...,...
395,20,2022-07-28,4,1,2v3,Meghan,"{3, 2}",Trial 20 Winner,NaN
396,21,2022-07-28,4,2,1v4,Meghan,"{4, 1}",Trial 20 Winner,NaN
397,26,2022-07-28,5,3,2v3,Isabel,"{3, 2}",Trial 20 Winner,13.0
398,33,2022-07-28,6,2,1v4,Isabel,"{4, 1}",Trial 20 Winner,19.0


- Dropping all rows that don't contain Winner

In [22]:
melted_reward_competition_df = melted_reward_competition_df[melted_reward_competition_df["Trial"].str.contains("Winner")]

In [23]:
melted_reward_competition_df = melted_reward_competition_df.dropna(subset="Winner")

In [24]:
melted_reward_competition_df["trial_number"] = melted_reward_competition_df["Trial"].apply(lambda x: int(x.lower().strip("trial").strip("winner")))

In [25]:
melted_reward_competition_df = melted_reward_competition_df.sort_values(["index", "trial_number"])

In [26]:
melted_reward_competition_df.head()

,index,Date,Cage,Box,Match,Scorer,animal_ids,Trial,Winner,trial_number
10,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 1 Winner,0:37-0:44,1
30,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 2 Winner,1:31-1:38,2
50,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 3 Winner,2:31-2:38,3
70,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 4 Winner,3:24-3:30,4
90,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 5 Winner,4:14-4:21,5


In [27]:
melted_reward_competition_df["trial_number"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [28]:
melted_reward_competition_df["winner_id"] = melted_reward_competition_df["Winner"].astype(str).apply(lambda x: x.split(".")[0])

In [36]:
melted_reward_competition_df["Winner"].unique()

array(['0:37-0:44', '1:31-1:38', '2:31-2:38', '3:24-3:30', '4:14-4:21',
       '5:20-5:27', '6:24-6:31', '7:23-7:30', '8:17-8:23', '9:17-9:23',
       '10:23-10:29', '11:23-11:30', '12:43-12:50', '13:43-13:50',
       '14:40-14:48', '15:39-15:46', '16:43-16:49', '18:02-18:09',
       '20:36-20:43', 18.0, '0:36-0:43', '1:29-1:38', '02:29-02:38',
       '3:22-3:29', '4:12-4:19', '5:19-5:26', '6:22-6:30', '7:23-7:29',
       '8:15-8:22', '9:15-9:23', '10:22-10:29', '11:22-11:29',
       '12:42-12:50', '13:42-13:50', '14:39-14:47', '15:38-15:47',
       '16:41-16:49', '18:01-18:08', '18:55-19:01', 13.0, '1:30-1:36',
       '2:29-2:36', '3:22-3:28', '4:12-4:18', '5:18-5:25', '6:22-6:28',
       '7:22-7:29', '9:15-9:21', '10:22-10:28', '11:21-11:28',
       '12:41-12:47', '13:41-13:48', '14:38-14:44', '15:38-15:44',
       '16:40-16:47', '18:00-18:07', '18:56-19:00', 10.0, '1:30- 1:37',
       '4:14-4:20', '10:23-10:30', '12:43-12:49', '14:40-14:46',
       '15:39-15:45', '16:42-16:49', '18:

## Add a column for the loser ID

In [33]:
melted_reward_competition_df.columns

Index(['index', 'Date', 'Cage', 'Box', 'Match', 'Scorer', 'animal_ids',
       'Trial', 'Winner', 'trial_number', 'winner_id'],
      dtype='object')

In [31]:
melted_reward_competition_df["animal_ids"].unique()

TypeError: unhashable type: 'set'

In [34]:
melted_reward_competition_df["loser_id"] = melted_reward_competition_df.apply(lambda x: (list(x["animal_ids"] - set(x["winner_id"]))), axis=1)

In [35]:
melted_reward_competition_df

,index,Date,Cage,Box,Match,Scorer,animal_ids,Trial,Winner,trial_number,winner_id,loser_id
10,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 1 Winner,0:37-0:44,1,0:37-0:44,[1]
30,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 2 Winner,1:31-1:38,2,1:31-1:38,[4]
50,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 3 Winner,2:31-2:38,3,2:31-2:38,[4]
70,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 4 Winner,3:24-3:30,4,3:24-3:30,[1]
90,3,2022-07-28,1,4,1v4,Isabel,"{4, 1}",Trial 5 Winner,4:14-4:21,5,4:14-4:21,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
319,36,2022-07-29,1,3,1v3,Naeliz,"{3, 1}",Trial 16 Winner,15:40-15:48,16,15:40-15:48,[3]
339,36,2022-07-29,1,3,1v3,Naeliz,"{3, 1}",Trial 17 Winner,16:45-16:51,17,16:45-16:51,[3]
359,36,2022-07-29,1,3,1v3,Naeliz,"{3, 1}",Trial 18 Winner,18:05-18:11,18,18:05-18:11,[3]
379,36,2022-07-29,1,3,1v3,Naeliz,"{3, 1}",Trial 19 Winner,18:58-19:04,19,18:58-19:04,[3]


In [29]:
melted_reward_competition_df["loser_id"] = melted_reward_competition_df.apply(lambda x: (list(x["animal_ids"] - set(x["winner_id"]))[0]), axis=1)

IndexError: list index out of range

In [ ]:
melted_reward_competition_df.head()

In [30]:
melted_reward_competition_df["winner_id"].unique()

array(['0:37-0:44', '1:31-1:38', '2:31-2:38', '3:24-3:30', '4:14-4:21',
       '5:20-5:27', '6:24-6:31', '7:23-7:30', '8:17-8:23', '9:17-9:23',
       '10:23-10:29', '11:23-11:30', '12:43-12:50', '13:43-13:50',
       '14:40-14:48', '15:39-15:46', '16:43-16:49', '18:02-18:09',
       '20:36-20:43', '18', '0:36-0:43', '1:29-1:38', '02:29-02:38',
       '3:22-3:29', '4:12-4:19', '5:19-5:26', '6:22-6:30', '7:23-7:29',
       '8:15-8:22', '9:15-9:23', '10:22-10:29', '11:22-11:29',
       '12:42-12:50', '13:42-13:50', '14:39-14:47', '15:38-15:47',
       '16:41-16:49', '18:01-18:08', '18:55-19:01', '13', '1:30-1:36',
       '2:29-2:36', '3:22-3:28', '4:12-4:18', '5:18-5:25', '6:22-6:28',
       '7:22-7:29', '9:15-9:21', '10:22-10:28', '11:21-11:28',
       '12:41-12:47', '13:41-13:48', '14:38-14:44', '15:38-15:44',
       '16:40-16:47', '18:00-18:07', '18:56-19:00', '10', '1:30- 1:37',
       '4:14-4:20', '10:23-10:30', '12:43-12:49', '14:40-14:46',
       '15:39-15:45', '16:42-16:49', '18:

In [ ]:
melted_reward_competition_df["loser_id"].unique()

In [ ]:
melted_reward_competition_df.head(n=25)

In [ ]:
raise ValueError("Stop!")

## Adding the session number

- Each new session has the date as a row, so we will get the index of all the rows with dates

In [ ]:
reward_competition_df.dropna(axis=0, subset="Date")

- Getting the indexes of where each new session starts

In [ ]:
reward_competition_df.dropna(axis=0, subset="Date").index

In [ ]:
session_indexes = reward_competition_df.dropna(axis=0, subset="Date").index

- Adding a column for the session number and then filling all the `NaNs`

In [ ]:
reward_competition_df = eloscore.add_session_number_column(reward_competition_df, session_indexes)
reward_competition_df = reward_competition_df.fillna(method='ffill')

- Making another column for plotting

In [ ]:
reward_competition_df = eloscore.add_session_number_column(reward_competition_df, session_indexes, session_number_column="session_number_plotting")

In [ ]:
reward_competition_df.head()

## Calculating ELO Score

- Example calculation

In [ ]:
eloscore.calculate_elo_score(subject_elo_score=1000, agent_elo_score=2000)

## Get the Elo score for all the events

In [ ]:
reward_competition_df.head()

In [ ]:
eloscore.update_elo_score(winner_id="A", loser_id="B")

- Function that creates a dictionary that has the original/updated elo score for each event

In [ ]:
index_to_elo_score_and_meta_data = eloscore.iterate_elo_score_calculation_for_data_frame(data_frame=reward_competition_df, winner_column="winner", loser_column="loser", additional_columns=["Date", "session_number", "session_number_plotting"])

In [ ]:
index_to_elo_score_and_meta_data[0]

- Turning the dictionary into a dataframe

In [ ]:
reward_competition_elo_score_df = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

In [ ]:
reward_competition_elo_score_df.head()

In [ ]:
reward_competition_elo_score_df.tail()

In [ ]:
reward_competition_elo_score_df.groupby("subject_id").count()

In [ ]:
reward_competition_elo_score_df.groupby("session_number").count()

# Plotting the Elo Score by match number

In [ ]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

In [ ]:
fig, ax = plt.subplots()

# Drawing vertical lines that represent when each session begins
for index, row in reward_competition_elo_score_df.dropna(subset=["session_number_plotting"]).iterrows():
    # Offsetting by 0.5 to avoid drawing the line on the dot
    plt.vlines(x=[row["total_match_number"] - 0.5], ymin=700, ymax=1300, colors='black', linestyle='dashed')

# Drawing a line for each subject
for subject in sorted(reward_competition_elo_score_df["subject_id"].unique()):
    # Getting all the rows with the current subject
    subject_df = reward_competition_elo_score_df[reward_competition_elo_score_df["subject_id"] == subject]
    # Making the dates into days after the first session by subtracting all the dates by the first date
    plt.plot(subject_df["total_match_number"], subject_df["updated_elo_score"], '-o', label=subject)

# Labeling the X/Y Axis and the title
ax.set_xlabel("Trial Number")
ax.set_ylabel("Elo Score")
# ax.set_title("Elo Score for Home Cage Observation: Cage {}".format(cage))
# To show the legend
ax.legend()
plt.xticks(rotation=90)
plt.ylim(700, 1300) 
plt.show()



# Calculate the Elo Score for all sheets

In [ ]:
xls = pd.ExcelFile("./data/Reward_Competition_Scoring_MasterList.xlsx")
sheets = xls.sheet_names

In [ ]:
sheets

In [ ]:
sheet_to_data_frame = defaultdict(dict)
for sheet in sheets[1:]:
    sheet_to_data_frame[sheet]["recording_data_frame"] = pd.read_excel("./data/Reward_Competition_Scoring_MasterList.xlsx", sheet_name=sheet, header=1).copy()
    # Adding the session numbers as a column
    session_indexes = sheet_to_data_frame[sheet]["recording_data_frame"].dropna(axis=0, subset="Date").index
    sheet_to_data_frame[sheet]["recording_data_frame"] = eloscore.add_session_number_column(sheet_to_data_frame[sheet]["recording_data_frame"], session_indexes)
    sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].fillna(method='ffill')
    sheet_to_data_frame[sheet]["recording_data_frame"] = eloscore.add_session_number_column(sheet_to_data_frame[sheet]["recording_data_frame"], session_indexes, session_number_column="session_number_plotting")

    # Splitting all the rows with commas in the ID
    sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].astype(str)
    sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].astype(str)
    
    sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].str.split(",")
    sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].explode("winner").reset_index(drop=True)
    sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].str.split(",")
    sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].explode("loser").reset_index(drop=True)

    sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].astype(float).astype(int).astype(str)
    sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].astype(float).astype(int).astype(str)
    
    # Calculating the Elo Score    
    index_to_elo_score_and_meta_data = eloscore.iterate_elo_score_calculation_for_data_frame(data_frame=sheet_to_data_frame[sheet]["recording_data_frame"], \
                                                                                                               winner_column="winner", loser_column="loser", additional_columns=["Date", "session_number", "session_number_plotting"])
    sheet_to_data_frame[sheet]["elo_score_data_frame"] = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [ ]:
sheets[1]

In [ ]:
earliest_date = str(sheet_to_data_frame[sheets[1]]["recording_data_frame"]["Date"].min()).split()[0]

In [ ]:
latest_date = str(sheet_to_data_frame[sheets[1]]["recording_data_frame"]["Date"].max()).split()[0]

In [ ]:
earliest_date

In [ ]:
latest_date

- Getting the cage numbers

In [ ]:
all_cages = "_".join(sorted([sheet.lower().strip("cage").strip() for sheet in sheets[1:]]))

In [ ]:
all_cages

In [ ]:
output_directory = "./proc/plots/reward_competition_elo_score/cage_{}_date_{}_{}".format(all_cages, earliest_date, latest_date)

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
for key, value in sheet_to_data_frame.items():
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_score_data_frame = value["elo_score_data_frame"]
    fig, ax = plt.subplots()

    # Drawing vertical lines that represent when each session begins
    for index, row in elo_score_data_frame.dropna(subset=["session_number_plotting"]).iterrows():
        # Offsetting by 0.5 to avoid drawing the line on the dot
        plt.vlines(x=[row["total_match_number"] - 0.5], ymin=600, ymax=1400, colors='black', linestyle='dashed')

    # Drawing a line for each subject
    for subject in sorted(elo_score_data_frame["subject_id"].astype(str).unique()):
        # Getting all the rows with the current subject
        subject_df = elo_score_data_frame[elo_score_data_frame["subject_id"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        plt.plot(subject_df["total_match_number"], subject_df["updated_elo_score"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo Score")
    ax.set_title("Elo Score for Home Cage Observation: {}".format(key))
    # To show the legend
    ax.legend()
    plt.xticks(rotation=90)
    plt.ylim(600, 1400) 
    
    
    file_name = "reward_competition_elo_score_{}_date_{}_{}.png".format(key, earliest_date, latest_date)
    plt.savefig(os.path.join(output_directory, file_name))
    
    plt.show()